# Classification Problems: Final Project

Today we are going to answer the question: **What factors determine the willingness to stay with the current employer? Predict
whether or not an employee will quit their job.**

This problem is also sometimes called **employee churn**

Before we start, please, the following packages: 
- numpy
- pandas
- matplotlib.pyplot
- seaborn

In addition, for classifications we previously needed the following functions from sklearn: 
- train_test_split from sklearn.model_selection
- LogisticRegression from sklearn.linear_model
- confusion_matrix, accuracy_score, precision_score, recall_score, f1_score from sklearn.metrics
- DecisionTreeClassifier from sklearn.tree
- KNeighborsClassifier from sklearn.neighbors
- RandomForestClassifier, AdaBoostClassifier from sklearn.ensemble

In [237]:
# import packages here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

We are going to be using the data from 4 datasets today: 

- **attrition.csv** contains employee number and observed target (this is what we are going to be using as y). Please, notice, that this dataset does not contain all the customer number which are present in df1 and df2, initial right answers are available just for the part of employees
- **df1.csv** and **df2.csv** contain employee number and corresponding employee characteristics

The last dataset we are going to use at the end of the project. 
- **sample_output.csv** this dataset contains target for the rest of the employees. 


The goal of the task is to build a model, using attrition.csv, df1.csv and df2.csv which will help us to get right predictions for the employees in the sample_output.csv

In [238]:
# read the data from attrition.csv, df1.csv, df2.csv
attr = pd.read_csv('attrition.csv')
df1 = pd.read_csv('df1.csv')
df2 = pd.read_csv('df2.csv')
so = pd.read_csv('sample_output.csv')


In [239]:
so.describe()

,EmployeeNumber,Attrition
count,147.000000,147.0
mean,100073.000000,1.0
std,42.579338,0.0
min,100000.000000,1.0
25%,100036.500000,1.0
50%,100073.000000,1.0
75%,100109.500000,1.0
max,100146.000000,1.0


In [240]:
# example: we know that Employee 104 did not quit the job. Lets see what the characteristics of this person were and use it in training

display(attr[attr['EmployeeNumber'] == 104])
display(df2[df2['EmployeeNumber'] == 104])
display(df1[df1['EmployeeNumber'] == 104])

,EmployeeNumber,Attrition
0,104,No


,EmployeeNumber,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,YearlyIncome
0,104,Y,Yes,12.0,3.0,3.0,80.0,2.0,10.0,1.0,2.0,10.0,8.0,3.0,0.0,No,61512.0


,EmployeeNumber,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked
0,104,30.0,Travel_Rarely,852.0,Research & Development,1.0,1.0,Life Sciences,1.0,4.0,Male,55.0,2.0,2.0,Laboratory Technician,4.0,Married,5126.0,15998.0,1.0


In [241]:
# example: we want to predict if the Employee 100000 is going to quit the job. For that we use it's characteristics and the model we developed.
# Employee 100000 is not part of the training process

display(attr[attr['EmployeeNumber'] == 100000])
display(df2[df2['EmployeeNumber'] == 100000])
display(df1[df1['EmployeeNumber'] == 100000])
display(so[so['EmployeeNumber'] == 100000])

,EmployeeNumber,Attrition
1323,100000,NaN


,EmployeeNumber,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,YearlyIncome
1323,100000,Y,No,11.0,3.0,4.0,80.0,0.0,6.0,3.0,2.0,6.0,5.0,1.0,1.0,NaN,30708.0


,EmployeeNumber,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked
1323,100000,35.0,Travel_Rarely,1343.0,Research & Development,27.0,1.0,Medical,1.0,3.0,Female,53.0,2.0,1.0,Research Scientist,1.0,Single,2559.0,17852.0,1.0


,EmployeeNumber,Attrition
0,100000,1


Here are the descriptions for some columns: 

<img src="img/desc1.png">
<img src="img/desc2.png">
<img src="img/desc3.png">

## 1. Initial Data Analysis

In [242]:
# please check how many rows and columns each dataset contains, how do first rows look, 
# what are the data formats, how many unique values each of the variables have, if there are missings
attr.shape, df1.shape, df2.shape

((4449, 2), (4449, 20), (4449, 17))

In [243]:
df1.head().T

,0,1,2,3,4
EmployeeNumber,104,1638,164,395,53
Age,30.0,38.0,26.0,28.0,35.0
BusinessTravel,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely
DailyRate,852.0,397.0,841.0,1117.0,464.0
Department,Research & Development,Research & Development,Research & Development,Research & Development,Research & Development
DistanceFromHome,1.0,2.0,6.0,8.0,4.0
Education,1.0,2.0,3.0,2.0,2.0
EducationField,Life Sciences,Medical,Other,Life Sciences,Other
EmployeeCount,1.0,1.0,1.0,1.0,1.0
EnvironmentSatisfaction,4.0,4.0,3.0,4.0,3.0


In [244]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4449 entries, 0 to 4448
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EmployeeNumber            4449 non-null   int64  
 1   Over18                    4449 non-null   object 
 2   OverTime                  4449 non-null   object 
 3   PercentSalaryHike         4449 non-null   float64
 4   PerformanceRating         4449 non-null   float64
 5   RelationshipSatisfaction  4449 non-null   float64
 6   StandardHours             4449 non-null   float64
 7   StockOptionLevel          4449 non-null   float64
 8   TotalWorkingYears         4449 non-null   float64
 9   TrainingTimesLastYear     4449 non-null   float64
 10  WorkLifeBalance           4449 non-null   float64
 11  YearsAtCompany            4449 non-null   float64
 12  YearsInCurrentRole        4449 non-null   float64
 13  YearsSinceLastPromotion   4449 non-null   float64
 14  YearsWit

In [245]:
df1.nunique()

EmployeeNumber             4449
Age                         179
BusinessTravel                3
DailyRate                   886
Department                    3
DistanceFromHome            158
Education                     5
EducationField                6
EmployeeCount                 1
EnvironmentSatisfaction       4
Gender                        2
HourlyRate                   71
JobInvolvement                4
JobLevel                      5
JobRole                       9
JobSatisfaction               4
MaritalStatus                 3
MonthlyIncome              1349
MonthlyRate                1427
NumCompaniesWorked           10
dtype: int64

## 2. Data Preparation

In [246]:
# create one dataset out of df1 and df2 using pd.merge function
# more about this https://pandas.pydata.org/docs/user_guide/merging.html#joining-on-index
# or https://www.datacamp.com/tutorial/joining-dataframes-pandas
# or here is a similar problem described: https://stackoverflow.com/questions/41463119/join-two-dataframes-on-common-column-in-python
df = pd.merge(df1, df2, how='inner', on='EmployeeNumber')
df.T


,0,1,2,3,4,5,6,7,8,9,...,4439,4440,4441,4442,4443,4444,4445,4446,4447,4448
EmployeeNumber,104,1638,164,395,53,1467,727,351,555,253,...,7970,7971,7972,7973,7974,7975,7976,7977,7978,7979
Age,30.0,38.0,26.0,28.0,35.0,34.0,32.0,42.0,34.0,40.0,...,41.0,42.0,42.0,42.0,8823.0,8823.0,8823.0,44.0,44.0,44.0
BusinessTravel,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Frequently,Travel_Rarely,...,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely,Travel_Rarely,Non-Travel,Non-Travel,Non-Travel
DailyRate,852.0,397.0,841.0,1117.0,464.0,1107.0,1018.0,269.0,296.0,989.0,...,582.0,1396.0,1396.0,1396.0,621.0,621.0,621.0,381.0,381.0,381.0
Department,Research & Development,Research & Development,Research & Development,Research & Development,Research & Development,Human Resources,Research & Development,Research & Development,Sales,Research & Development,...,Research & Development,Research & Development,Research & Development,Research & Development,Research & Development,Research & Development,Research & Development,Research & Development,Research & Development,Research & Development
DistanceFromHome,1.0,2.0,6.0,8.0,4.0,9.0,3.0,2.0,6.0,4.0,...,28.0,6.0,6.0,6.0,15.0,15.0,15.0,918785.0,918785.0,918785.0
Education,1.0,2.0,3.0,2.0,2.0,4.0,2.0,3.0,2.0,1.0,...,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
EducationField,Life Sciences,Medical,Other,Life Sciences,Other,Technical Degree,Life Sciences,Medical,Marketing,Medical,...,Life Sciences,Medical,Medical,Medical,Medical,Medical,Medical,Medical,Medical,Medical
EmployeeCount,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EnvironmentSatisfaction,4.0,4.0,3.0,4.0,3.0,1.0,3.0,4.0,4.0,4.0,...,1.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0


In [247]:
# filter out all the rows where 'attrition' is not filled
df.dropna(subset='Attrition', inplace=True)
df


,EmployeeNumber,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,YearlyIncome
0,104,30.0,Travel_Rarely,852.0,Research & Development,1.0,1.0,Life Sciences,1.0,4.0,...,2.0,10.0,1.0,2.0,10.0,8.0,3.0,0.0,No,61512.0
1,1638,38.0,Travel_Rarely,397.0,Research & Development,2.0,2.0,Medical,1.0,4.0,...,1.0,10.0,6.0,4.0,5.0,4.0,0.0,2.0,No,93072.0
2,164,26.0,Travel_Rarely,841.0,Research & Development,6.0,3.0,Other,1.0,3.0,...,0.0,5.0,3.0,2.0,5.0,4.0,4.0,3.0,No,28416.0
3,395,28.0,Travel_Rarely,1117.0,Research & Development,8.0,2.0,Life Sciences,1.0,4.0,...,0.0,5.0,3.0,3.0,5.0,3.0,0.0,2.0,No,39720.0
4,53,35.0,Travel_Rarely,464.0,Research & Development,4.0,2.0,Other,1.0,3.0,...,1.0,1.0,3.0,3.0,1.0,0.0,0.0,0.0,No,23412.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4444,7975,8823.0,Travel_Rarely,621.0,Research & Development,15.0,3.0,Medical,1.0,1.0,...,1.0,10.0,2.0,3.0,10.0,7.0,0.0,5.0,Yes,95736.0
4445,7976,8823.0,Travel_Rarely,621.0,Research & Development,15.0,3.0,Medical,1.0,1.0,...,1.0,10.0,2.0,3.0,10.0,7.0,0.0,5.0,Yes,95736.0
4446,7977,44.0,Non-Travel,381.0,Research & Development,918785.0,3.0,Medical,1.0,1.0,...,0.0,9.0,5.0,3.0,5.0,2.0,1.0,4.0,Yes,44496.0
4447,7978,44.0,Non-Travel,381.0,Research & Development,918785.0,3.0,Medical,1.0,1.0,...,0.0,9.0,5.0,3.0,5.0,2.0,1.0,4.0,Yes,44496.0


In [248]:
# if some datatypes are incorrect change them
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4302 entries, 0 to 4448
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EmployeeNumber            4302 non-null   int64  
 1   Age                       4302 non-null   float64
 2   BusinessTravel            4302 non-null   object 
 3   DailyRate                 4302 non-null   float64
 4   Department                4302 non-null   object 
 5   DistanceFromHome          4302 non-null   float64
 6   Education                 4302 non-null   float64
 7   EducationField            4302 non-null   object 
 8   EmployeeCount             4302 non-null   float64
 9   EnvironmentSatisfaction   4302 non-null   float64
 10  Gender                    4302 non-null   object 
 11  HourlyRate                4302 non-null   float64
 12  JobInvolvement            4302 non-null   float64
 13  JobLevel                  4302 non-null   float64
 14  JobRole 

In [249]:
df.isnull().sum()

EmployeeNumber              0
Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

In [250]:
# Replace Yes and No with 1 and 0 in column Attrition
df['Attrition'] = (df['Attrition'] == 'Yes').astype(int)
#df['Attrition'].head(10)
df['Attrition'].nunique()

2

In [251]:
df['BusinessTravel'].value_counts()

Travel_Rarely        3041
Travel_Frequently     816
Non-Travel            445
Name: BusinessTravel, dtype: int64

In [252]:
# Encode categorical variables !!! which need encoding
# If you think the variable can be included as it is, motivate your decision and do not encode this variable

# create lists of numberix and categorical features
cat_features = df.select_dtypes(include=object).columns.tolist()
cat_features


['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [253]:
num_features = [
    'Education', 
'JobInvolvement', 
'EnvironmentSatisfaction', 
'JobSatisfaction',
'PerformanceRating',
'WorkLifeBalance',
'RelationshipSatisfaction'
]
num_features

['Education',
 'JobInvolvement',
 'EnvironmentSatisfaction',
 'JobSatisfaction',
 'PerformanceRating',
 'WorkLifeBalance',
 'RelationshipSatisfaction']

In [254]:
features = cat_features + num_features
features

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime',
 'Education',
 'JobInvolvement',
 'EnvironmentSatisfaction',
 'JobSatisfaction',
 'PerformanceRating',
 'WorkLifeBalance',
 'RelationshipSatisfaction']

In [276]:
enc_df = pd.get_dummies(df, columns=features)
pd.set_option('display.max_rows', None)
enc_df.head().T

,0,1,2,3,4
EmployeeNumber,104.0,1638.0,164.0,395.0,53.0
Age,30.0,38.0,26.0,28.0,35.0
DailyRate,852.0,397.0,841.0,1117.0,464.0
DistanceFromHome,1.0,2.0,6.0,8.0,4.0
EmployeeCount,1.0,1.0,1.0,1.0,1.0
HourlyRate,55.0,54.0,46.0,66.0,75.0
JobLevel,2.0,3.0,1.0,1.0,1.0
MonthlyIncome,5126.0,7756.0,2368.0,3310.0,1951.0
MonthlyRate,15998.0,14199.0,23300.0,4488.0,10910.0
NumCompaniesWorked,1.0,3.0,1.0,1.0,1.0


## 3. EDA for categorical features (exploratory data analysis)

In [267]:
# Values are 0 and 1 - it means it sums up all the 1 and divide it with all rows.
# it means we have average value about 0.52 - it is almost half the numbers are 1
global_attrition = np.mean(df['Attrition'])
global_attrition

0.5165039516503952

In [268]:
# Explore categorical variables by calculating attrition rates per category
for col in features:
    grouped_stats = df.groupby(col).Attrition.agg(['sum', 'count', 'mean'])
    grouped_stats['differens in mean'] = (grouped_stats['mean'] - global_attrition)*100
    display(grouped_stats)

# Sum - number of people who quit the job in each category
# Count - number of people in each category
# Mean - in each category, how many people quit the job on average
# example: Non-travelers - about 52% of the employees quit the job

,sum,count,mean,differens in mean
BusinessTravel,,,,
Non-Travel,231,445,0.519101,0.259717
Travel_Frequently,433,816,0.530637,1.413330
Travel_Rarely,1558,3041,0.512331,-0.417248


,sum,count,mean,differens in mean
Department,,,,
Human Resources,99,182,0.543956,2.745209
Research & Development,1429,2815,0.507638,-0.886630
Sales,694,1305,0.531801,1.529681


,sum,count,mean,differens in mean
EducationField,,,,
Human Resources,47,80,0.587500,7.099605
Life Sciences,895,1768,0.506222,-1.028223
Marketing,252,472,0.533898,1.739435
Medical,683,1344,0.508185,-0.831943
Other,132,245,0.538776,2.227156
Technical Degree,213,393,0.541985,2.548078


,sum,count,mean,differens in mean
Gender,,,,
Female,892,1707,0.522554,0.605024
Male,1330,2595,0.512524,-0.397987


,sum,count,mean,differens in mean
JobRole,,,,
Healthcare Representative,196,384,0.510417,-0.608728
Human Resources,84,150,0.560000,4.349605
Laboratory Technician,378,755,0.500662,-1.584170
Manager,147,287,0.512195,-0.430883
Manufacturing Director,191,417,0.458034,-5.847038
Research Director,116,229,0.506550,-0.995373
Research Scientist,469,883,0.531144,1.463988
Sales Executive,498,950,0.524211,0.770657
Sales Representative,143,247,0.578947,6.244342


...

,sum,count,mean,differens in mean
EnvironmentSatisfaction,,,,
1.0,444,842,0.527316,1.081196
2.0,421,812,0.518473,0.196895
3.0,713,1358,0.525037,0.853287
4.0,644,1290,0.499225,-1.727915


,sum,count,mean,differens in mean
JobSatisfaction,,,,
1.0,472,859,0.549476,3.297218
2.0,410,818,0.501222,-1.528146
3.0,666,1281,0.519906,0.340237
4.0,674,1344,0.501488,-1.501586


,sum,count,mean,differens in mean
PerformanceRating,,,,
3.0,1902,3672,0.517974,0.146990
4.0,320,630,0.507937,-0.856744


,sum,count,mean,differens in mean
WorkLifeBalance,,,,
1.0,144,242,0.595041,7.853737
2.0,501,976,0.513320,-0.318428
3.0,1340,2631,0.509312,-0.719190
4.0,237,453,0.523179,0.667486


,sum,count,mean,differens in mean
RelationshipSatisfaction,,,,
1.0,420,789,0.532319,1.581544
2.0,457,892,0.512332,-0.417211
3.0,680,1331,0.510894,-0.560989
4.0,665,1290,0.515504,-0.100008


In [294]:
significal_list = [
    'BusinessTravel_Travel_Frequently',
'Department_Human Resources',
'Department_Sales',
'EducationField_Human Resources',
'EducationField_Other',
'EducationField_Technical Degree',
'Gender_Female',
'JobRole_Human Resources',
'JobRole_Laboratory Technician',
'JobRole_Manager',
'JobRole_Manufacturing Director',
'JobRole_Sales Representative',
'OverTime_Yes',
'Education_1.0',
'JobInvolvement_1.0',
'JobSatisfaction_1.0',
'WorkLifeBalance_1.0',
'RelationshipSatisfaction_1.0'
]
print(len(significal_list))
print(significal_list)

18
['BusinessTravel_Travel_Frequently', 'Department_Human Resources', 'Department_Sales', 'EducationField_Human Resources', 'EducationField_Other', 'EducationField_Technical Degree', 'Gender_Female', 'JobRole_Human Resources', 'JobRole_Laboratory Technician', 'JobRole_Manager', 'JobRole_Manufacturing Director', 'JobRole_Sales Representative', 'OverTime_Yes', 'Education_1.0', 'JobInvolvement_1.0', 'JobSatisfaction_1.0', 'WorkLifeBalance_1.0', 'RelationshipSatisfaction_1.0']


## 4. EDA for numeric features

In [295]:
df_dropped_category_values = df.drop(features, axis=1)
df_dropped_category_values.nunique()


EmployeeNumber             4302
Age                         179
DailyRate                   886
DistanceFromHome            158
EmployeeCount                 1
HourlyRate                   71
JobLevel                      5
MonthlyIncome              1349
MonthlyRate                1427
NumCompaniesWorked           10
PercentSalaryHike            15
StandardHours                 1
StockOptionLevel              4
TotalWorkingYears           172
TrainingTimesLastYear         7
YearsAtCompany              166
YearsInCurrentRole          164
YearsSinceLastPromotion     153
YearsWithCurrManager        157
Attrition                     2
YearlyIncome               1349
dtype: int64

In [296]:
# Check correlations between numeric features and also with target
df_dropped_category_values[df_dropped_category_values.columns].corrwith(df_dropped_category_values['Attrition'])

EmployeeNumber             0.452997
Age                        0.086266
DailyRate                 -0.003835
DistanceFromHome           0.067157
EmployeeCount                   NaN
HourlyRate                -0.002821
JobLevel                  -0.024330
MonthlyIncome             -0.017704
MonthlyRate               -0.007690
NumCompaniesWorked        -0.001138
PercentSalaryHike         -0.003036
StandardHours                   NaN
StockOptionLevel          -0.035847
TotalWorkingYears          0.063771
TrainingTimesLastYear      0.011058
YearsAtCompany             0.067983
YearsInCurrentRole         0.077537
YearsSinceLastPromotion    0.076210
YearsWithCurrManager       0.084582
Attrition                  1.000000
YearlyIncome              -0.017704
dtype: float64

## 5. Train test split

In [321]:
X = pd.DataFrame(enc_df, columns=significal_list)
y = enc_df['Attrition']
X.head()

,BusinessTravel_Travel_Frequently,Department_Human Resources,Department_Sales,EducationField_Human Resources,EducationField_Other,EducationField_Technical Degree,Gender_Female,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Sales Representative,OverTime_Yes,Education_1.0,JobInvolvement_1.0,JobSatisfaction_1.0,WorkLifeBalance_1.0,RelationshipSatisfaction_1.0
0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0


In [347]:
X_all = enc_df.drop(['EmployeeNumber','Attrition'], axis=1)
X_all.head()

,Age,DailyRate,DistanceFromHome,EmployeeCount,HourlyRate,JobLevel,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,YearlyIncome,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes,Education_1.0,Education_2.0,Education_3.0,Education_4.0,Education_5.0,JobInvolvement_1.0,JobInvolvement_2.0,JobInvolvement_3.0,JobInvolvement_4.0,EnvironmentSatisfaction_1.0,EnvironmentSatisfaction_2.0,EnvironmentSatisfaction_3.0,EnvironmentSatisfaction_4.0,JobSatisfaction_1.0,JobSatisfaction_2.0,JobSatisfaction_3.0,JobSatisfaction_4.0,PerformanceRating_3.0,PerformanceRating_4.0,WorkLifeBalance_1.0,WorkLifeBalance_2.0,WorkLifeBalance_3.0,WorkLifeBalance_4.0,RelationshipSatisfaction_1.0,RelationshipSatisfaction_2.0,RelationshipSatisfaction_3.0,RelationshipSatisfaction_4.0
0,30.0,852.0,1.0,1.0,55.0,2.0,5126.0,15998.0,1.0,12.0,80.0,2.0,10.0,1.0,10.0,8.0,3.0,0.0,61512.0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0
1,38.0,397.0,2.0,1.0,54.0,3.0,7756.0,14199.0,3.0,19.0,80.0,1.0,10.0,6.0,5.0,4.0,0.0,2.0,93072.0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1
2,26.0,841.0,6.0,1.0,46.0,1.0,2368.0,23300.0,1.0,19.0,80.0,0.0,5.0,3.0,5.0,4.0,4.0,3.0,28416.0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0
3,28.0,1117.0,8.0,1.0,66.0,1.0,3310.0,4488.0,1.0,21.0,80.0,0.0,5.0,3.0,5.0,3.0,0.0,2.0,39720.0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1
4,35.0,464.0,4.0,1.0,75.0,1.0,1951.0,10910.0,1.0,12.0,80.0,1.0,1.0,3.0,1.0,0.0,0.0,0.0,23412.0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0


In [348]:
X.isnull().sum()

BusinessTravel_Travel_Frequently    0
Department_Human Resources          0
Department_Sales                    0
EducationField_Human Resources      0
EducationField_Other                0
EducationField_Technical Degree     0
Gender_Female                       0
JobRole_Human Resources             0
JobRole_Laboratory Technician       0
JobRole_Manager                     0
JobRole_Manufacturing Director      0
JobRole_Sales Representative        0
OverTime_Yes                        0
Education_1.0                       0
JobInvolvement_1.0                  0
JobSatisfaction_1.0                 0
WorkLifeBalance_1.0                 0
RelationshipSatisfaction_1.0        0
dtype: int64

In [349]:
# split your dataset into train and test parts
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)

## 6. Modelling and evaluation

Think of the models we have learned about last weekend and apply some of them to your dataset. Play around with some hyperparameters of the models and see how your results change

In [350]:
model_logistic = LogisticRegression(penalty='l2')
model_logistic.fit(X_train, y_train)

LogisticRegression()

In [351]:
y_prediction_test = model_logistic.predict(X_test)
y_pred_train = model_logistic.predict(X_train)

In [352]:
accuracy_test_logistic = accuracy_score(y_test, y_prediction_test)
precision_test_logistic = precision_score(y_test, y_prediction_test)
recall_test_logistic = recall_score(y_test, y_prediction_test)
f1_test_logistic = f1_score(y_test, y_prediction_test)
accuracy_test_logistic, precision_test_logistic, recall_test_logistic,f1_test_logistic

(0.6631823461091754, 0.7008310249307479, 0.5816091954022988, 0.635678391959799)

In [353]:
accuracy_train = accuracy_score(y_train, y_pred_train)
precision_train = precision_score(y_train, y_pred_train)
recall_train = recall_score(y_train, y_pred_train)
f1_train = f1_score(y_train, y_pred_train)
accuracy_train, precision_train, recall_train, f1_train

(0.6256902063353676, 0.6724257083621286, 0.544487968662563, 0.6017316017316018)

In [354]:
X_all.columns

Index(['Age', 'DailyRate', 'DistanceFromHome', 'EmployeeCount', 'HourlyRate',
       'JobLevel', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'PercentSalaryHike', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'YearlyIncome', 'BusinessTravel_Non-Travel',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'Department_Human Resources', 'Department_Research & Development',
       'Department_Sales', 'EducationField_Human Resources',
       'EducationField_Life Sciences', 'EducationField_Marketing',
       'EducationField_Medical', 'EducationField_Other',
       'EducationField_Technical Degree', 'Gender_Female', 'Gender_Male',
       'JobRole_Healthcare Representative', 'JobRole_Human Resources',
       'JobRole_Laboratory Technician', 'JobRole_Manager',
       'JobRole_Manufacturing Director', 'J

In [355]:
model_logistic.coef_

array([[ 1.48622998e-04, -6.05084786e-05,  1.31372477e-06,
        -1.36155747e-07, -6.67861033e-06, -1.64151573e-07,
        -1.37499692e-07, -7.11263965e-06, -5.06569743e-07,
        -1.99730128e-06, -1.08924598e-05, -6.39199331e-07,
         1.37157257e-04, -4.97500919e-07,  1.66721924e-04,
         1.59361062e-04,  1.47035343e-04,  2.03514002e-04,
        -1.64999631e-06, -2.20630974e-08, -3.44333276e-08,
        -7.96593224e-08,  2.26271564e-08, -2.42126945e-07,
         8.33440412e-08,  3.00050440e-08, -1.00150640e-07,
        -8.49222902e-09, -2.01355872e-07,  5.47972698e-08,
         8.90406795e-08,  1.48734135e-08, -1.51029161e-07,
        -3.35023492e-08,  3.91177930e-08, -1.24791974e-07,
         6.64758651e-09, -1.56329245e-07,  5.24014524e-08,
        -2.96830309e-09,  4.13824714e-08,  4.18868200e-08,
        -1.85336105e-07, -6.34662385e-08,  1.12646596e-07,
        -1.36155747e-07, -4.55143059e-07,  3.18987311e-07,
         1.17584664e-07, -1.40943854e-07,  1.38477182e-0

In [356]:
list(zip(X_train.columns, model_logistic.coef_[0]))


[('Age', 0.00014862299760226883),
 ('DailyRate', -6.050847858889649e-05),
 ('DistanceFromHome', 1.3137247656941231e-06),
 ('EmployeeCount', -1.3615574739766895e-07),
 ('HourlyRate', -6.67861033144546e-06),
 ('JobLevel', -1.6415157270925e-07),
 ('MonthlyIncome', -1.3749969211893349e-07),
 ('MonthlyRate', -7.112639650617969e-06),
 ('NumCompaniesWorked', -5.065697433395613e-07),
 ('PercentSalaryHike', -1.9973012815514775e-06),
 ('StandardHours', -1.0892459791813234e-05),
 ('StockOptionLevel', -6.391993309103703e-07),
 ('TotalWorkingYears', 0.0001371572574508364),
 ('TrainingTimesLastYear', -4.975009192225604e-07),
 ('YearsAtCompany', 0.00016672192358851813),
 ('YearsInCurrentRole', 0.00015936106190960394),
 ('YearsSinceLastPromotion', 0.00014703534277640365),
 ('YearsWithCurrManager', 0.0002035140015252735),
 ('YearlyIncome', -1.6499963054099632e-06),
 ('BusinessTravel_Non-Travel', -2.2063097374501752e-08),
 ('BusinessTravel_Travel_Frequently', -3.443332762633571e-08),
 ('BusinessTravel_T

## 7. Choose the best model and final evaluation

You still have sample_output dataset untouched. Use the best model you have created and evaluate it's performance on this new dataset. Was the performance different from what you have expected?

In [357]:
# read sample_output.csv

In [262]:
# merge it with the features needed for the model

In [263]:
# apply the final chosen model and evaluate the results